In [11]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 2) # update this to your liking

headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

In [26]:
# begin code
url_links = []
property_metadata = defaultdict(dict)

# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=dateupdated-desc&page={page}" # a single page
    bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser") # makes bs object

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # complies RE string into RE expression 
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])

print(url_links[0:1])

['https://www.domain.com.au/5-75-gadd-street-northcote-vic-3070-16065924']


In [65]:
# for each url, scrape some basic metadata
for property_url in url_links[0:1]:
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    # may need preprocessing 
    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # finds latitude and longitude from integrated Google Map
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]
    
    
    rooms_info = bs_object.find("div", {"data-testid": "property-features"}).findAll("span", {"data-testid": "property-features-text-container"})
    for i in range(0, len(rooms_info)):
        attr_desc = str(rooms_info[i].text).split(' ')

        property_metadata[property_url][attr_desc[1]] = attr_desc[0]

    '''
    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

    '''

In [66]:
property_metadata

defaultdict(dict,
            {'https://www.domain.com.au/5-75-gadd-street-northcote-vic-3070-16065924': {'name': '5/75 Gadd Street Northcote VIC 3070',
              'cost_text': '$530 per week',
              'coordinates': [-37.76274900000001, 144.9897927],
              'rooms': ['2 Beds', '2 Baths', '2 Parking'],
              'Beds': '2',
              'Baths': '2',
              'Parking': '2'}})